# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [28]:
# your code here
#GroupBy it's the way
df = orders.groupby(['CustomerID']).sum().reset_index()


# Reset the index of the DataFrame, and use the default one instead. If the DataFrame has a MultiIndex, this method can remove one or more levels.
# Credit : 'https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html'
df_agr = orders.groupby('CustomerID')['amount_spent'].sum().reset_index().sort_values('amount_spent', ascending=False)
df_agr

,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06
...,...,...
4099,17956,12.75
3015,16454,6.90
1794,14792,6.20
3218,16738,3.75


In [32]:

#select customers aggregated amount_spent/ quantile range
# bins and labels for each bin
bin_labels = ['Others', 'Preferred', 'VIP']
df_agr['type_customer'] = pd.qcut(df_agr['amount_spent'], q=[0, .75, .95, 1], labels=bin_labels)
df_agr



,CustomerID,amount_spent,type_customer
1690,14646,280206.02,VIP
4202,18102,259657.30,VIP
3729,17450,194550.79,VIP
3009,16446,168472.50,VIP
1880,14911,143825.06,VIP
...,...,...,...
4099,17956,12.75,Others
3015,16454,6.90,Others
1794,14792,6.20,Others
3218,16738,3.75,Others


In [ ]:
#Subproblem 3 

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [41]:
condition = df_agr['type_customer']=='VIP' #Transform in equals the columns type costumer and vip from df_agr
#This is gonna be stored in other variable called condition
orders_VIP = df_agr[condition] #Store one variable with the result

#Merging the values in Orders_vip with orders for get the values of country column
orders_values = pd.merge(orders_VIP, orders, on='CustomerID', how='inner')

#drop duplicates
orders_values = orders_values.drop_duplicates('CustomerID')

#group by country - count
orders_values.groupby('Country')['CustomerID'].count().reset_index().sort_values('CustomerID', ascending=False)


,Country,CustomerID
16,United Kingdom,177
7,Germany,10
6,France,9
15,Switzerland,3
13,Spain,2
11,Portugal,2
0,Australia,2
4,EIRE,2
8,Japan,2
5,Finland,1


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [42]:
# your code here

In [43]:
condition = (df_agr['type_customer']=='VIP') | (df_agr['type_customer']=='Preferred') #OR |
orders_VIP=df_agr[condition]

#Same procedure but using other values from merge method

orders_values = pd.merge(orders_VIP, orders, on='CustomerID', how='inner')

orders_values = orders_values.drop_duplicates('CustomerID')

orders_values.groupby('Country')["CustomerID"].count().reset_index().sort_values('CustomerID', ascending=False)



,Country,CustomerID
26,United Kingdom,932
10,Germany,39
9,France,29
2,Belgium,11
25,Switzerland,9
23,Spain,7
21,Portugal,7
19,Norway,7
14,Italy,5
8,Finland,5
